# astroVIPER Installation Test

```python
conda create --name astroviper python=3.11 --no-default-packages
conda activate astroviper
#Only on mac: conda install -c conda-forge python-casacore
pip install astroviper
```

## Check installation

In [ ]:
import os

from importlib.metadata import version

try:
    import astroviper

    print('astroVIPER version', version('astroviper'), 'already installed.')
except ImportError as e:
    print(e)
    print('Installing astroVIPER')

    os.system("pip install astroviper")

    import xradio
    print('astroVIPER version', version('astroviper'), ' installed.')

## Optional if casarc does not have path to casadata

In [ ]:
'''
import pkg_resources
casa_data_dir = pkg_resources.resource_filename('casadata', '__data__')
rc_file = open(os.path.expanduser("~/.casarc"), "a+")  # append mode
rc_file.write("\n measures.directory: " + casa_data_dir)
rc_file.close()
'''

## Setup Dask Cluster

In [ ]:
import astroviper
from astroviper.client import local_client

viper_client = local_client(cores=2, memory_limit="4GB", worker_log_parms=None)
viper_client

## Download and convert dataset

In [ ]:
from xradio.data.datasets import download

infile = "Antennae_North.cal.lsrk.split.ms"
outfile = "Antennae_North.cal.lsrk.split.vis.zarr"
download(file="Antennae_North.cal.lsrk.split.ms",source='dropbox')

from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme="ddi_intent_field"

chunks_on_disk = {'frequency':3}
convert_msv2_to_processing_set(infile=infile, outfile=outfile, 
                               partition_scheme=partition_scheme,
                               parallel=False,overwrite=True,
                               chunks_on_disk=chunks_on_disk)


## Inspect Processing Set

In [ ]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set
#sel_parms = {'xds_id':[0,1,2]}
#fields = ["NGC4038 - Antennae North"]
intents = ["OBSERVE_TARGET#ON_SOURCE"]
fields = None
ps = read_processing_set(ps_name,
                        intents=intents,
                        fields=fields)
ps.summary()

## Inspect a single ms_v4

In [ ]:
ms_xds = ps[list(ps.keys())[0]] 

## Creating Parallel Coordinates for the Map

### Example: Frequency Parallel Coordinate

In [ ]:
from astroviper._concurrency._graph_tools import _make_parallel_coord

parallel_coords = {}
n_chunks=3
parallel_coords['frequency'] = _make_parallel_coord(coord=ms_xds.frequency, n_chunks=n_chunks)
parallel_coords['frequency']

## Create a chunk function and map graph

In [ ]:
from astroviper._concurrency._graph_tools import _map
import dask

def my_func(input_parms):
    print('Keys in input parameters: ',list(input_parms.keys()))
    print('Test_input:', input_parms['test_input'])
    print('data_sel',input_parms['data_sel'])
    print('chunk_coords',input_parms['chunk_coords'])
    print('chunk_indices',input_parms['chunk_indices'])
    print('chunk_id',input_parms['chunk_id'])
    print('*****'*10)
    return input_parms['test_input']

#['test_input', 'input_data_name', 'viper_local_dir', 'date_time', 'data_sel', 'chunk_coords', 'chunk_indx', 'chunk_id', 'parallel_dims']
input_parms= {}
input_parms['test_input'] = 42

sel_parms = {}
sel_parms["fields"] = ["NGC4038 - Antennae North"]
sel_parms["intents"] = ["OBSERVE_TARGET#ON_SOURCE"]
graph = _map(
    input_data_name = ps_name,
    input_data_type = 'processing_set',
    ps_sel_parms=sel_parms,
    parallel_coords=parallel_coords,
    func_chunk=my_func,
    input_parms=input_parms,
    client=None,
)

dask.visualize(graph)

## Run the graph

In [ ]:
dask.compute(graph)

## Adding a reduce

In [ ]:
from astroviper._concurrency._graph_tools import _reduce

def my_sum(input_1, input_2, input_parms):
    return input_1 + input_2 + input_parms['test_input']

input_parms= {}
input_parms['test_input'] = 5
graph_reduce = _reduce(graph, my_sum, input_parms)
dask.visualize(graph_reduce)

## Run the graph

In [ ]:
dask.compute(graph_reduce) 